In [1]:
from run import *

In [2]:
valid_pd = pd.read_csv('/home/zhangkechi/workspace/riddle/dataset/valid.csv')
valid_pd.head()

,A,Q
0,逝,着手咫尺间
1,逝,折不断，扣不住。
2,逝,折不断，丢不还
3,逝,折了不扣，丢了不还
4,袂,但求温暖，不求遂心快意


In [3]:
output_pkl = '/home/zhangkechi/workspace/riddle/clone_detection/code/run_bert-base-chinese_chaizi/1654030807.3463356eval_vecs.pkl'

In [4]:
with open(output_pkl,'rb') as f:
    tmp = pickle.load(f)

In [5]:
tmp['vecs'].shape, tmp['ans_vecs'].shape, tmp['labels'].shape

((5480, 768), (5480, 768), (5480,))

In [6]:
scores= np.matmul(tmp['vecs'],tmp['ans_vecs'].T)
sort_ids=np.argsort(scores, axis=-1, kind='quicksort', order=None)[:,::-1]
index2ans = list(valid_pd['A'])
index2ans = {i:index2ans[i] for i in range(len(index2ans))}
predict_words = []
for i in tqdm(range(sort_ids.shape[0])):
    this_predict = []
    this_ids = sort_ids[i]
    for each_id in this_ids:
        each_label = index2ans[each_id]
        if each_label not in this_predict:
            this_predict.append(each_label)
    predict_words.append(this_predict)

100%|███████████████████████████████████████████████████████████████████████████| 5480/5480 [04:02<00:00, 22.60it/s]


In [10]:
# scores[0]
# scores[0][sort_ids[0]]
# sort_ids[0]

In [7]:
mrr_index = []
ground_truth = list(valid_pd['A'])
for i in range(len(ground_truth)):
    mrr_index.append(predict_words[i].index(ground_truth[i]))

In [13]:
len(mrr_index)

5480

In [12]:
len([e for e in mrr_index if e < 10])

1615

In [9]:
len(predict_words[0])

1457

In [10]:
mrr_score = [1/(e + 1) for e in mrr_index]
mrr_score = sum(mrr_score) / len(mrr_score)
print(mrr_score)

0.17505883559491522


In [23]:
def mrr_calculate(vecs, ans_vecs, ground_truth,MRR_num = 100):
        scores= np.matmul(vecs,ans_vecs.T)
        sort_ids=np.argsort(scores, axis=-1, kind='quicksort', order=None)[:,::-1]
        index2ans = ground_truth
        index2ans = {i:index2ans[i] for i in range(len(index2ans))}
        predict_words = []
        for i in range(sort_ids.shape[0]):
            this_predict = []
            this_pred_num = 0
            this_ids = sort_ids[i]
            for each_id in this_ids:
                each_label = index2ans[each_id]
                if each_label not in this_predict:
                    this_predict.append(each_label)
                    this_pred_num += 1
                    if this_pred_num > MRR_num:
                        break
            predict_words.append(this_predict)
        mrr_index = []
        for i in range(len(ground_truth)):
            if ground_truth[i] in predict_words[i]:
                mrr_index.append(predict_words[i].index(ground_truth[i]))
            else:
                mrr_index.append(100000)
            # mrr_index.append(predict_words[i].index(ground_truth[i]))
        mrr_score = [1/(e + 1) for e in mrr_index]
        mrr_score = sum(mrr_score) / len(mrr_score)
        return mrr_score
mrr_calculate(tmp['vecs'], tmp['ans_vecs'], list(valid_pd['A']))

0.16500264535195613

In [24]:
file2score = {}
root_path = '/home/zhangkechi/workspace/riddle/clone_detection/code/run_bert-base-chinese_chaizi/'
for each_path in tqdm(os.listdir(root_path)):
    if each_path.endswith('vecs.pkl'):
        this_path = os.path.join(root_path, each_path)
        with open(this_path,'rb') as f:
            tmp = pickle.load(f)
        mrr_score = mrr_calculate(tmp['vecs'], tmp['ans_vecs'], list(valid_pd['A']))
        file2score[each_path] = mrr_score
for k in file2score:
    print(k, file2score[k])

100%|█████████████████████████████████████████████████████████████████████████████| 105/105 [05:16<00:00,  3.01s/it]

1654045418.0644822eval_vecs.pkl 0.1658956320155989
1654043699.4347608eval_vecs.pkl 0.16152989659743325
1654046275.5794492eval_vecs.pkl 0.15950767313004027
1654040832.8157375eval_vecs.pkl 0.17044401687528446
1654017607.431044eval_vecs.pkl 0.015401335891580941
1654020768.8873668eval_vecs.pkl 0.10496269850713513
1654041979.565728eval_vecs.pkl 0.1630236707513619
1654040546.5434382eval_vecs.pkl 0.1687131522059434
1654042553.1133084eval_vecs.pkl 0.16116721971013379
1654043412.6293762eval_vecs.pkl 0.163946651370078
1654027082.7790415eval_vecs.pkl 0.17016134266794164
1654032238.6865556eval_vecs.pkl 0.1692538459679733
1654028515.9036777eval_vecs.pkl 0.1654594105439483
1654017321.6663573eval_vecs.pkl 0.018115243577040578
1654033958.1531172eval_vecs.pkl 0.16407313638718196
1654023353.9835236eval_vecs.pkl 0.15877832660546679
1654025648.0891845eval_vecs.pkl 0.1700042235233847
1654023640.8057785eval_vecs.pkl 0.15538020506503505
1654046848.2730129eval_vecs.pkl 0.15647028982433206
1654037970.1649358ev

In [25]:
file2score_pair = list(file2score.items())

In [29]:
file2score_pair.sort(key=lambda x:-x[1])

In [31]:
file2score_pair[:10]

[('1654030807.3463356eval_vecs.pkl', 0.17360303006722083),
 ('1654027655.350034eval_vecs.pkl', 0.17130011143378676),
 ('1654028229.1687155eval_vecs.pkl', 0.17097276290654545),
 ('1654035964.438939eval_vecs.pkl', 0.17087299510268514),
 ('1654039687.5798843eval_vecs.pkl', 0.1707428141822356),
 ('1654040832.8157375eval_vecs.pkl', 0.17044401687528446),
 ('1654028802.0196233eval_vecs.pkl', 0.1703046524520733),
 ('1654027082.7790415eval_vecs.pkl', 0.17016134266794164),
 ('1654025648.0891845eval_vecs.pkl', 0.1700042235233847),
 ('1654026796.0927563eval_vecs.pkl', 0.16949586702582922)]